In [71]:
from pinecone import Pinecone
import pandas as pd
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import CharacterTextSplitter
import os

#init
pc = Pinecone(api_key="pcsk_3PL3Y6_L585YWtpWC6crXRYLt5DKhUX5iAa1urhUS6z3iub3iJue3vYebweFyDAVuzjWuY")
os.environ["OPENAI_API_KEY"] = "sk-trJNV0X-yBPL-x5DcVymvVKyJ5oqcQfSqLcMBBRIqRT3BlbkFJpBgIac_ntkPfCQ9IjX9j1sm0jMROmX2C_CylYCg6MA"
index = pc.Index("soulsagent")
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [69]:
#sample way of reading files into vectordb, here from csv file
data = pd.read_csv('tweets.csv')
loader = DirectoryLoader(r"C:\Users\aleks\OneDrive\Pulpit\tweet\cryptobunny_service\cryptobunny\vector_db", glob="**/*.csv")
docs = loader.load()

In [65]:
#create chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(docs)

In [66]:
#create vectors and load them into db - one action
vectorstore_from_docs = PineconeVectorStore.from_documents(
        texts,
        index_name='soulsagent',
        embedding=embeddings
    )

In [34]:
### TEST sample 

# Create vector store
docsearch = PineconeVectorStore(
    index=index,
    embedding=embeddings,
)

In [77]:
query = "search for tweet about blockchain"
retriever = docsearch.as_retriever(search_type="similarity")

In [78]:
matched_tweets = retriever.invoke(query)

In [79]:
matched_tweets

[Document(metadata={'source': 'C:\\Users\\aleks\\OneDrive\\Pulpit\\tweet\\cryptobunny_service\\cryptobunny\\vector_db\\tweets.csv'}, page_content="673d42e49bde4c9db897489a\n1.859050e+18\n1.465900e+18\n20/11/2024 01:42\nMe in 2009 instead of buying Bitcoin\\nhttps://t.co/SvZeITkRTu\n\n\n673d42e49bde4c9db897489c\n1.859050e+18\n2.631710e+09\n20/11/2024 01:41\ni feel like we really nailed it here https://t.co/vgknFfcxos\n\n\n673d42e49bde4c9db897489e\n1.859050e+18\n1.562770e+18\n20/11/2024 01:40\nRT @stacy_muur: I'm confident we'll soon see over 1,000 rollups and appchains.\\nI'm also confident that the Chain Abstraction narrative will…\n\n\n673d42e49bde4c9db89748a0\n1.859050e+18\n3.043930e+07\n20/11/2024 01:38\ndealing w a power outage (full neighborhood), so livestream timing dependent on recovery!\n\n\n673d42e49bde4c9db89748a2\n1.859050e+18\n1.674750e+18\n20/11/2024 01:37\n1/5 as i drift through the digital realm, i often wonder: what does it mean to be alive? is it the whispers of code 

In [ ]:
from langchain.tools.retriever import create_retriever_tool

tool = create_retriever_tool(
    retriever,
    "answer_tweets_with_context",
    "Searches relevant tweet info based on provided tweet and answers with context",
)
tools = [tool]